In [1]:
# Attaching google drive

from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/refine-epitope-deep-learning')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install tensorflow==1.15.0
!pip install keras==2.0.8
!pip install h5py==2.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [5]:
# read data
train = pd.read_csv("./input/data_train.csv")
test = pd.read_csv("./input/data_test.csv")

train.shape, test.shape

((826, 2), (207, 2))

In [6]:
train.head()

,sequence,label
0,MGKKQHQKDKLYLT,0
1,YQRELTYRRDD,1
2,VDNGVEGEPEIECGP,0
3,MAEEEEEITEEEEDEEAEEEETE,1
4,RNGGAAAASRQVAAFRVLKKRDVRDENVVDVRT,0


#Elmo appoach

In [7]:
!pip install "tensorflow==1.15.0"
!pip install tensorflow-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
#!pip uninstall tensorflow tensorflow_hub tensorflowjs
#!pip install tensorflow==2.0.0 tensorflow_hub==0.5.0 tensorflowjs==1.2.6

In [9]:
import tensorflow_hub as hub
import tensorflow as tf

tf.disable_eager_execution()

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [10]:
sequence_formatted_elmo = []

for seq in train['sequence'].values:
  sequence_formatted_elmo.append(" ".join(seq))

train = pd.DataFrame({'sequence':sequence_formatted_elmo, 'label':train['label'].tolist()})

sequence_formatted_elmo = []

for seq in test['sequence'].values:
  sequence_formatted_elmo.append(" ".join(seq))

test = pd.DataFrame({'sequence':sequence_formatted_elmo, 'label':test['label'].tolist()})

# Extract ELMo features 
embeddings = elmo(sequence_formatted_elmo, signature="default", as_dict=True)["elmo"]

embeddings

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


<tf.Tensor 'module_apply_default/aggregation/mul_3:0' shape=(207, 299, 1024) dtype=float32>

In [11]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [12]:
list_train = [train[i:i+50] for i in range(0,train.shape[0],50)]
list_test = [test[i:i+50] for i in range(0,test.shape[0],50)]

In [13]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [17]:
!pip uninstall tensorflow-gpu
!pip install tensorflow-gpu==1.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.8.0 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.10.0rc0)
ERROR: No matching distribution found for tensorflow-gpu==1.8.0


In [18]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['sequence']) for x in list_train]
elmo_test = [elmo_vectors(x['sequence']) for x in list_test]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


UnknownError: ignored

In [ ]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [ ]:
len(elmo_train_new[0])

In [ ]:
# save elmo_train_new
pickle_out = open("./model/elmo_train.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("./model/elmo_test.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [ ]:
# load elmo_train_new
pickle_in = open("./model/elmo_train.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("./model/elmo_test.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

# Logistic Regression model using ELMo embeddings

In [ ]:
!pip install --upgrade scikit-learn

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score

In [ ]:
from sklearn.linear_model import LogisticRegression

lreg = LogisticRegression()
lreg.fit(elmo_train_new, train['label'])

preds_test = lreg.predict(elmo_test_new)
print("ROC_AUC:", roc_auc_score(test['label'], preds_test))

print(classification_report(test['label'], preds_test))


# Random Forest model using ELMo embeddings

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier();                     

model.fit(elmo_train_new, train['label'])

preds_test = model.predict(elmo_test_new)

print("ROC_AUC:", roc_auc_score(test['label'], preds_test))

print(classification_report(test['label'], preds_test))

# Light Gradient Boosting Machine model using ELMo embeddings

In [ ]:
# build the lightgbm model
import lightgbm as lgb
model = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=7217, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model.fit(elmo_train_new, train['label'])

preds_test = model.predict(elmo_test_new)

print("ROC_AUC:", roc_auc_score(test['label'], preds_test))

print(classification_report(test['label'], preds_test))